In [4]:
from dataclasses import dataclass
from typing import Tuple, Dict, Generator,Callable,Optional
import numpy as np
import matplotlib.pyplot as plt
import itertools
import os
os.chdir("..")
from rl.markov_process import MarkovRewardProcess,FiniteMarkovProcess,MarkovProcess
from rl.markov_process import FiniteMarkovRewardProcess
from rl.markov_process import Transition,TransitionStep,ReturnStep
from rl.markov_process import RewardTransition
from rl.distribution import Constant,Categorical,SampledDistribution,Distribution
from rl.gen_utils.common_funcs import get_logistic_func

# p1



In [43]:
gamma=1
V0=np.array([10,1,0])
Q1_s1=np.array([8,10])+gamma*np.array([[0.2,0.6,0.2],[0.1,0.2,0.7]])@V0
Q1_s2=np.array([1,-1])+gamma*np.array([[0.3,0.3,0.4],[0.5,0.3,0.2]])@V0
V1=np.array([Q1_s1.max(),Q1_s2.max(),0])
print("Iteration 1:","V1:",V1,"Q0_s1",Q0_s1,"Q0_s2",Q0_s2)
print("greedy_policy1: s1 do a2; s2 indifferent.")

Q1_s1=np.array([8,10])+gamma*np.array([[0.2,0.6,0.2],[0.1,0.2,0.7]])@V1
Q1_s2=np.array([1,-1])+gamma*np.array([[0.3,0.3,0.4],[0.5,0.3,0.2]])@V1
V2=np.array([Q2_s1.max(),Q2_s2.max(),0])
print("Iteration 2:","V2:",V2,"Q2_s1",Q2_s1,"Q2_s2",Q2_s2)
print("greedy_policy2: s1 do a1; s2 do a2.")


Iteration 1: V1: [11.2  4.3  0. ] Q0_s1 [10.6 11.2] Q0_s2 [4.3 4.3]
greedy_policy1: s1 do a2; s2 indifferent.
Iteration 2: V2: [14.098  7.177  0.   ] Q2_s1 [14.098 12.46 ] Q2_s2 [6.613 7.177]
greedy_policy2: s1 do a1; s2 do a2.


To be honest, I don't see an obvious reason why greedy_policy in iteration2 will just be the optimal policy. To me, it seems like the optimal policy is simply to stay away from terminating state s3 and it's a self-reinforcing process?

# p2



In [48]:
# Will do it later since already covered in practice midterm

# p3



In [49]:
# Will do it later since already covered in practice midterm

# p4



I didn't write out the entire code, but I guess you can see where I'm heading to. The brute force method I'm using doesn't look like efficient enough to me. Is there a better method, such as an interaction between two SimpleInventoryMDP Objects?

In [64]:
from dataclasses import dataclass
from typing import Tuple, Dict
from rl.markov_decision_process import FiniteMarkovDecisionProcess
from rl.markov_decision_process import FinitePolicy, StateActionMapping
from rl.markov_process import FiniteMarkovProcess, FiniteMarkovRewardProcess
from rl.distribution import Categorical, Constant
from scipy.stats import poisson


@dataclass(frozen=True)
class InventoryState:
    on_hand: int
    on_order: int
    def inventory_position(self) -> int:
        return self.on_hand + self.on_order


InvOrderMapping = StateActionMapping[InventoryState, int]


class TwoInventoryMDPCap(FiniteMarkovDecisionProcess[InventoryState, int]):

    def __init__(
        self,
        capacity_1: int,
        poisson_lambda_1: float,
        holding_cost_1: float,
        stockout_cost_1: float,
        capacity_2: int,
        poisson_lambda_2: float,
        holding_cost_2: float,
        stockout_cost_2: float,
        K1: int,
        K2: int
    ):
        self.capacity_1: int= capacity_1
        self.poisson_lambda_1: float=poisson_lambda_1
        self.holding_cost_1: float=holding_cost_1
        self.stockout_cost_1: float=stockout_cost_1
        self.capacity_2: int=capacity_2
        self.poisson_lambda_2: float=poisson_lambda_2
        self.holding_cost_2: float=holding_cost_2
        self.stockout_cost_2: float=stockout_cost_2
        self.K1: int=K1
        self.K2: int=K2
        self.poisson_distr_1 = poisson(poisson_lambda_1)
        self.poisson_distr_2 = poisson(poisson_lambda_2)
        super().__init__(self.get_action_transition_reward_map())

    def get_action_transition_reward_map(self) -> InvOrderMapping:
        """
        state: two InventoryStates; action:(supplier_order_1,supplier_order_2,transport(from 1 to 2)),
        reward:-(holding_cost_1+stock_out_cost_1+supplier_trans_1+holding_cost_2+stock_out_cost_2+supplier_trans_2+trans_cost)
        """
        d: Dict[Tuple[InventoryState,InventoryState], Dict[Tuple[int,int,int], Categorical[Tuple[Tuple[InventoryState,InventoryState],
                                                            float]]]] = {}
        
        for on_hand_1 in range(self.capacity_1 + 1):
            for on_hand_2 in range(self.capacity_2 + 1):
                for on_order_1 in range(self.capacity_1 + 1 - on_hand_1):
                    for on_order_2 in range(self.capacity_2 + 1 - on_hand_2):
                        k_max=min(on_hand_1,self.capacity_2-on_hand_2-on_order_2)
                        k_min=-min(on_hand_2,self.capacity_1-on_hand_1-on_order_1)
                        for trans in range(k_min,k_max):
                            # Here the problem isn't clear enough. I assume the transportation between stores
                            #happens immediately. This definition determines how we calculate overnight holding cost.
                            state_1: InventoryState = InventoryState(on_hand_1-trans, on_order_1)
                            state_2: InventoryState = InventoryState(on_hand_2+trans, on_order_2)
                            ip_1: int = state_1.inventory_position()
                            ip_2: int = state_2.inventory_position()
                            base_reward_1: float = - self.holding_cost_1 * state_1.on_hand
                            base_reward_2: float = - self.holding_cost_2 * state_2.on_hand
                            d1: Dict[Tuple[int,int,int], Categorical[Tuple[Tuple[InventoryState,InventoryState], float]]] = {}
                            # loop order1 and order2
                            for order1 in range(self.capacity_1 - ip_1 + 1):
                                for order2 in range(self.capacity_2 - ip_2 + 1):
                                    sr_probs_dict: Dict[Tuple[Tuple[InventoryState,InventoryState], float]] =\
                                    {((InventoryState(ip_1 - i, order1),InventoryState(ip_2 - j, order2)), \
                                    base_reward):self.poisson_distr_1.pmf(i)*self.poisson_distr_2.pmf(j) \
                                      for i,j in zip(range(ip_1),range(ip_2))}

                        d1[(order1,order2,trans)] = Categorical(sr_probs_dict)

                        d[(InventoryState(on_hand_1, on_order_1),InventoryState(on_hand_2, on_order_2))] = d1
        return d



In [66]:
new_simulation=TwoInventoryMDPCap(
        capacity_1=2,
        poisson_lambda_1=1,
        holding_cost_1=1,
        stockout_cost_1=10,
        capacity_2=2,
        poisson_lambda_2=2,
        holding_cost_2=1,
        stockout_cost_2=10,
        K1=1,
        K2=10)

UnboundLocalError: local variable 'sr_probs_dict' referenced before assignment